# Keras Tutorial

## Import Utilities & Dependencies

- `print_function` works in Python 2 and Python 3
- Keras uses the NumPy mathematics library to manipulate arrays and matrices. Matplotlib is a plotting library for NumPy: you'll use it to inspect a training data item.
- Import Keras 2.0.6 and the components needed for the model. [FutureWarning due to NumPy 1.14](https://github.com/h5py/h5py/issues/961)
- Import coremltools

In [ ]:
from __future__ import print_function
from matplotlib import pyplot as plt

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

import coremltools
# coremltools supports Keras version 2.0.6
print('keras version ', keras.__version__)

## Load & Pre-Process Data

### Training & Validation Data Sets

`mnist.load_data()` downloads from https://s3.amazonaws.com/img-datasets/mnist.npz — this takes a little while.

In [ ]:
# First, get your data!
(x_train, y_train), (x_val, y_val) = mnist.load_data()

### Inspect x & y Data

In [ ]:
# Inspect x data
print('x_train shape: ', x_train.shape)
# (60000, 28, 28)
print(x_train.shape[0], 'training samples')
# 60000 train samples
print('x_val shape: ', x_val.shape)
# (10000, 28, 28)
print(x_val.shape[0], 'validation samples')
# 10000 validation samples

print('First x sample\n', x_train[0])
# An array of 28 arrays, each containing 28 gray-scale values between 0 and 255
# Plot first x sample
plt.imshow(x_train[0])
plt.show()

# Inspect y data
print('y_train shape: ', y_train.shape)
# (60000,)
print('First 10 y_train elements:', y_train[:10])
# [5 0 4 1 9 2 1 3 1 4]

### Set input & output dimensions

MNIST data items are 28 x 28-pixel images, and you want to classify each as a digit between 0 and 9.

`x_train.shape` is an array of 3 elements: number of data samples, number of rows of each data sample, number of columns of each data sample

In [ ]:
img_rows, img_cols = x_train.shape[1], x_train.shape[2]
num_classes = 10

### Reshape x Data & Set Input Shape

- Insert the channels, either before or after the image's rows and columns. MNIST data samples are gray-scale, so the number of channels is 1.
- Set the input shape of the sample data, with the channels at the correct end.

In [ ]:
# Set input_shape for channels_first or channels_last
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_val = x_val.reshape(x_val.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

### Inspect Reshaped x Data

TensorFlow image data format is channels-last.

In [ ]:
print('x_train shape:', x_train.shape)
# x_train shape: (60000, 28, 28, 1)
print('x_val shape:', x_val.shape)
# x_val shape: (10000, 28, 28, 1)
print('input_shape:', input_shape)
# input_shape: (28, 28, 1)

### Convert Data Type & Normalize Values

MNIST image data values are of type `uint8`, in the range [0, 255], but Keras needs values of type `float32`, in the range [0, 1].

In [ ]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_val /= 255

### Inspect Normalized x Data

In [ ]:
print('First x sample, normalized\n', x_train[0])
# An array of 28 arrays, each containing 28 arrays, each with one value between 0 and 1

### Reformat y Data

`y_train` is a 1-dimensional array with 60000 elements, but the model needs a 60000 x 10 matrix to represent the 10 categories.

**Note:** Run this cell **once only**! Running it again will produce incorrect results.

In [ ]:
print('y_train shape: ', y_train.shape)
# (60000,)
print('First 10 y_train elements:\n', y_train[:10])
# [5 0 4 1 9 2 1 3 1 4]
# Convert 1-dimensional class arrays to 10-dimensional class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_val = np_utils.to_categorical(y_val, num_classes)

### Inspect Reformatted y Data

`y_train` is now an array of 10-element arrays, each containing all zeros except at the index that the image matches.

In [ ]:
print('New y_train shape: ', y_train.shape)
# (60000, 10)
print('First 10 y_train elements, reshaped:\n', y_train[:10])
# An array of 10 arrays, each with 10 elements, 
# all zeros except at index 5, 0, 4, 1, 9 etc.

## Define  Model Architecture

### [Malireddi's Architecture](https://sriraghu.com/2017/07/06/computer-vision-in-ios-coremlkerasmnist/)

In [ ]:
model_m = Sequential()
model_m.add(Conv2D(32, (5, 5), input_shape=input_shape, activation='relu'))
model_m.add(MaxPooling2D(pool_size=(2, 2)))
model_m.add(Dropout(0.5))
model_m.add(Conv2D(64, (3, 3), activation='relu'))
model_m.add(MaxPooling2D(pool_size=(2, 2)))
model_m.add(Dropout(0.2))
model_m.add(Conv2D(128, (1, 1), activation='relu'))
model_m.add(MaxPooling2D(pool_size=(2, 2)))
model_m.add(Dropout(0.2))
model_m.add(Flatten())
model_m.add(Dense(128, activation='relu'))
model_m.add(Dense(num_classes, activation='softmax'))
# Inspect model's layers, output shapes, number of trainable parameters
print(model_m.summary())

### [Chollet's Architecture](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py)

## Train the Model

### Define Callbacks List

In [ ]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]

### Compile & Fit Model

- On a MacBook Pro, this step takes approximately 15 minutes. Reducing `batch_size` or increasing `epochs` will increase the run time.
- You can run this cell more than once, to improve the model's accuracy.
- To *manually* stop early, click the stop button in the toolbar.

In [ ]:
model_m.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

# Hyper-parameters
batch_size = 200
epochs = 10

# Enable validation to use ModelCheckpoint and EarlyStopping callbacks.
model_m.fit(
    x_train, y_train, batch_size=batch_size, epochs=epochs,
    callbacks=callbacks_list, validation_data=(x_val, y_val), verbose=1)

## Convert to Core ML Model

In [ ]:
output_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
# For the first argument, use the filename of the newest .h5 file in the notebook folder.
coreml_mnist = coremltools.converters.keras.convert(
    'best_model.06-0.06.h5', input_names=['image'], output_names=['output'], 
    class_labels=output_labels, image_input_names='image')

### Inspect Core ML model

Check the input type is `imageType`, not multi array

In [ ]:
print(coreml_mnist)

### Add Metadata for Xcode

Substitute your own name and license info for the first two items

In [ ]:
coreml_mnist.author = 'raywenderlich.com'
coreml_mnist.license = 'Razeware'
coreml_mnist.short_description = 'Image based digit recognition (MNIST)'
coreml_mnist.input_description['image'] = 'Digit image'
coreml_mnist.output_description['output'] = 'Probability of each digit'
coreml_mnist.output_description['classLabel'] = 'Labels of digits'

### Save the Core ML Model

In [ ]:
coreml_mnist.save('MNISTClassifier.mlmodel')